In [1]:
import datetime
import Image
import gc
import numpy as np
import os
import random
from scipy import misc
import string
import time
import sklearn.metrics as skm

# Set some Theano config before initializing
os.environ["THEANO_FLAGS"] = "mode=FAST_RUN,device=cpu,floatX=float32,allow_gc=False,openmp=True"
import theano

import matplotlib
matplotlib.use('Agg')
from matplotlib import pyplot as plt

import dwdii_bc_model_helper as bc
import bc_models as models

random.seed(20275)

/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scipy.linalg.blas instead
  DeprecationWarning)
/usr/lib/python2.7/dist-packages/scipy/lib/_util.py:35: DeprecationWarning: Module scipy.linalg.blas.fblas is deprecated, use scip

In [2]:
print "device:", theano.config.device
print "floatX:",  theano.config.floatX
print "mode:", theano.config.mode
print "openmp:", theano.config.openmp
print "allow_gc:", theano.config.allow_gc

device: cpu
floatX: float32
mode: FAST_RUN
openmp: True
allow_gc: False


In [3]:
imagePath = "/root/bc_data/ddsm-png.25"
#imagePath = "/root/bc_data/Data_Thresholded/DDSM"
trainDataPath = "../../data/ddsm_train.csv"
testDataPath = "../../data/ddsm_test.csv"
imgResize = (150, 150)

In [4]:
os.listdir('../../data')

/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.
  def _singleton_printers_default(self):
/usr/local/lib/python2.7/dist-packages/IPython/core/formatters.py:672: DeprecationWarning: PlainTextFormat

['ddsm_test.csv', 'ddsm_train.csv', 'ddsm_val.csv', 'mias_all.csv']

# Load Training and Test Data

In this section, the training/validation data is loaded. The load_data function pre-balances the data set by removing images from over-represented emotion classes.

### Training Data

In [5]:
metaData, meta2, mCounts = bc.load_training_metadata(trainDataPath, balanceViaRemoval=True, verbose=True)

Raw Balance
----------------
benign 541
malignant 852
normal 2793
balanaceViaRemoval.avgE: 1395
balanaceViaRemoval.theshold: 1395.0

After Balancing
----------------
benign 541
malignant 852
normal 929


In [6]:
meta2[meta2.keys()[0]]

'0'

In [7]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_data, Y_data = bc.load_data(trainDataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_data.shape
print Y_data.shape

Raw Balance
----------------
benign 541
malignant 852
normal 2793
balanaceViaRemoval.avgE: 1395
balanaceViaRemoval.theshold: 1395.0

After Balancing
----------------
benign 541
malignant 852
normal 929
0: A_0526_1.LEFT_CC.LJPEG.png


dwdii_bc_model_helper.py:160: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  X_data = np.zeros([total, x, y])
dwdii_bc_model_helper.py:161: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y_data = np.zeros([total, 1], dtype=np.int8)


0.02153316106804478897502153316: B_3515_1.RIGHT_MLO.LJPEG.png
0.04306632213608957795004306632: B_3623_1.RIGHT_MLO.LJPEG.png
0.06459948320413436692506459948: B_3411_1.RIGHT_MLO.LJPEG.png
0.08613264427217915590008613264: A_0515_1.RIGHT_CC.LJPEG.png
0.1076658053402239448751076658: A_1054_1.RIGHT_MLO.LJPEG.png
0.1291989664082687338501291990: A_0241_1.LEFT_MLO.LJPEG.png
0.1507321274763135228251507321: C_0015_1.RIGHT_MLO.LJPEG.png
0.1722652885443583118001722653: C_0160_1.RIGHT_MLO.LJPEG.png
0.1937984496124031007751937984: B_3506_1.RIGHT_MLO.LJPEG.png
0.2153316106804478897502153316: B_3024_1.RIGHT_MLO.LJPEG.png
0.2368647717484926787252368648: B_3076_1.LEFT_MLO.LJPEG.png
0.2583979328165374677002583979: A_0010_1.RIGHT_CC.LJPEG.png
0.2799310938845822566752799311: B_3021_1.LEFT_CC.LJPEG.png
0.3014642549526270456503014643: A_1030_1.LEFT_MLO.LJPEG.png
0.3229974160206718346253229974: A_0573_1.RIGHT_CC.LJPEG.png
0.3445305770887166236003445306: B_3514_1.RIGHT_CC.LJPEG.png
0.366063738156761412575366063

### Load Test Set 

In [8]:
# Actually load some representative data for model experimentation
maxData = len(metaData)
X_test, Y_test = bc.load_data(testDataPath, imagePath, maxData = maxData, verboseFreq = 50, imgResize=imgResize)
print X_test.shape
print Y_test.shape

Raw Balance
----------------
benign 142
malignant 206
normal 699
balanaceViaRemoval.avgE: 349
balanaceViaRemoval.theshold: 349.0

After Balancing
----------------
benign 142
malignant 206
normal 232
0: B_3380_1.RIGHT_MLO.LJPEG.png


dwdii_bc_model_helper.py:160: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  X_data = np.zeros([total, x, y])
dwdii_bc_model_helper.py:161: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  Y_data = np.zeros([total, 1], dtype=np.int8)


0.02153316106804478897502153316: A_0470_1.RIGHT_MLO.LJPEG.png
0.04306632213608957795004306632: A_1025_1.LEFT_MLO.LJPEG.png
0.06459948320413436692506459948: C_0020_1.RIGHT_MLO.LJPEG.png
0.08613264427217915590008613264: A_0265_1.LEFT_MLO.LJPEG.png
0.1076658053402239448751076658: B_3677_1.LEFT_CC.LJPEG.png
0.1291989664082687338501291990: A_0594_1.LEFT_MLO.LJPEG.png
0.1507321274763135228251507321: C_0110_1.RIGHT_CC.LJPEG.png
(371, 150, 150)
(371, 1)


## Split Training/Test Sets
The following code segment splits the data into training and test data sets. Currently this is a standard 80/20 split for training and test respectively after performing a random shuffle using the unison_shuffled_copies help method.

In [9]:
X_train = X_data
Y_train = Y_data

In [10]:
print X_train.shape
print X_test.shape

print Y_train.shape
print Y_test.shape

(1528, 150, 150)
(371, 150, 150)
(1528, 1)
(371, 1)


In [11]:
import collections
def yDist(y):
    bcCounts = collections.defaultdict(int)
    for a in range(0, y.shape[0]):
        bcCounts[y[a][0]] += 1
    return bcCounts

print "Y_train Dist: " + str(yDist(Y_train))
print "Y_test Dist: " + str(yDist(Y_test))


Y_train Dist: defaultdict(<type 'int'>, {0: 739, 1: 50, 2: 739})
Y_test Dist: defaultdict(<type 'int'>, {0: 180, 1: 12, 2: 179})


## Define and Load Trained Model

In [12]:
# Load the bc array for our count in the model definition
bcTypes = bc.bcNumerics()
print bcTypes
print len(bcTypes)

{'benign': 1, 'malignant': 2, 'normal': 0}
3


In [13]:
# Construct the model using our help function
model = models.bc_model_v0(len(bcTypes), verbose=True, 
                                        input_shape=(1,X_train.shape[1],X_train.shape[2]))

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
convolution2d_1 (Convolution2D)    (None, 32, 143, 143)2080        convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 32, 143, 143)0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)      (None, 32, 71, 71)  0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)    (None, 32, 67, 67)  25632       maxpooling2d_1[0][0]             
___________________________________________________________________________________________

## Training the Model

The following code segment trains the model using the run_network helper function. 

In [14]:
loadWeights = True
if loadWeights:
    model.load_weights("dwdii-bc-150-v0-Cloud-20170327.hdf5")

In [15]:
# Reshape to the appropriate shape for the CNN input
testX = X_test.reshape(X_test.shape[0], 1, X_test.shape[1],X_test.shape[2])
trainX = X_train.reshape(X_train.shape[0], 1, X_train.shape[1],X_train.shape[2])

In [ ]:
print "Training start: " + str(datetime.datetime.now())
m, h = models.run_network([trainX, testX, Y_train, Y_test], model, batch=50, epochs=30, verbosity=1)

In [ ]:
model.save_weights("dwdii-bc-150-v0-Thresholded-661-20170328.hdf5", overwrite=True)

### Experiment Results

#### Raw DDSM Images

Initial results based on "normal" being masked as "benign":
* bc_model_v0 (150x150, 800/200): 182s - loss: 0.0560 - acc: 0.9813 - val_loss: 1.9918 - val_acc: 0.6800
* bc_model_v0 (150x150, 2000/500): 473s - loss: 0.0288 - acc: 0.9925 - val_loss: 1.4040 - val_acc: 0.7260
   * somewhat balanced, Y_train Dist {0: 1223, 1: 777}, Y_test Dist: {0: 321, 1: 179}

Revised with "normal", "benign" and "malignant" labeled seperately:
* bc_model_v0 (150x150, 1311/328): 298s - loss: 0.0411 - acc: 0.9786 - val_loss: 1.3713 - val_acc: 0.6616

After creating fixed "train", "test" and "validate" data sets, using "train" and "test" as well as including the DDSM Benign cases:
* bc_model_v0 (150x150, 1554/363, 03.27.2017): 264s - loss: 0.0512 - acc: 0.9730 - val_loss: 1.3120 - val_acc: 0.6116

#### Thresholded Images

Using the "Data_Thresholded" images
* bc_model_v0 (150x150, Thresholded, 661/171, 03.28.2017): 124s - loss: 0.0529 - acc: 0.9743 - val_loss: 1.4331 - val_acc: 0.4971

In [ ]:
resultsValAcc = {}
resultsValAcc["1"] = 0.6800
resultsValAcc["2"] = 0.7260
resultsValAcc["3"] = 0.6616
resultsValAcc["03-27-2017"] = 0.6116
import dwdii_test as dwdii
#cmp = matplotlib.colors.Colormap("Blues")
dwdii.barChart(resultsValAcc, filename="../../figures/daniels_results_valacc.png", title="DDSM Results Acc_Val", yAxisLabel="val_acc %")

### Analyze Predictions with Test Set

In [17]:
predictOutput = model.predict(testX, batch_size=32, verbose=1)

371/371 [==============================] - 18s    


In [21]:
predClass = np.array(predictOutput[0]).argmax()
numBC = bc.numericBC()
numBC[predClass]

'normal'

In [34]:
Y_test[0][0]

2

In [36]:
predClasses = []
for i in range(len(predictOutput)):

    arPred = np.array(predictOutput[i])
    predictionProb = arPred.max()
    predictionNdx = arPred.argmax()
    predClassName = numBC[predictionNdx]
    predClasses.append(predictionNdx)

    print "{0}: {1} ({2})".format(i, predClassName, predictionProb)

0: normal (0.967189311981)
1: malignant (0.583556592464)
2: normal (1.0)
3: normal (0.980319678783)
4: normal (0.840614557266)
5: normal (0.999983072281)
6: normal (0.99999910593)
7: malignant (0.783349812031)
8: malignant (0.958830654621)
9: malignant (0.998692810535)
10: malignant (0.967710316181)
11: normal (0.5743470788)
12: normal (0.998214900494)
13: normal (1.0)
14: normal (0.736483097076)
15: normal (0.808021068573)
16: malignant (0.971956670284)
17: malignant (0.967767775059)
18: normal (0.995480358601)
19: normal (0.999975204468)
20: malignant (0.970632076263)
21: malignant (0.828493595123)
22: normal (0.566996335983)
23: malignant (0.998683989048)
24: normal (0.999999821186)
25: normal (0.998538255692)
26: malignant (0.546469807625)
27: normal (0.95605134964)
28: malignant (0.986320614815)
29: normal (0.930714964867)
30: normal (0.748172879219)
31: malignant (0.967880368233)
32: malignant (0.919455647469)
33: malignant (0.594031274319)
34: malignant (0.99745965004)
35: norma

In [46]:

cnf_matrix = skm.confusion_matrix(Y_test, predClasses)
cnf_matrix

array([[128,   0,  52],
       [  6,   0,   6],
       [ 81,   3,  95]])

In [44]:
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [61]:
class_names = numBC.values()
np.set_printoptions(precision=2)

In [56]:
# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')


Confusion matrix, without normalization
[[128   0  52]
 [  6   0   6]
 [ 81   3  95]]


In [58]:
plt.savefig('../../figures/confusion_matrix.png')

In [62]:
# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')
plt.savefig('../../figures/confusion_matrix_norm.png')

Normalized confusion matrix
[[ 0.71  0.    0.29]
 [ 0.5   0.    0.5 ]
 [ 0.45  0.02  0.53]]
